Consider the n-dimensional linear system of ordinary differential equations:
$$x'(t)=A(t)x(t)$$
where in addition, the matrix $A(t)$ is periodic with period $T$, i.e. $A(t+T)=A(t)$ for all real values of $t$.
Let $Φ(t)$ denote the principal fundamental matrix solution such that the columns
of $Φ(t)$ are linearly independent, and $Φ(0)=I$. Then, Floquet's theorem decomposes the principal
fundamental matrix as the product of a periodic matrix $P(t)$ with period $T$ and an exponential matrix $e^{tB}$.



In [2]:
using Plots, ApproxFun

In [3]:
function Base.expm{T<:Number}(A::Matrix{T},f::Fun)
    D,V = eig(A)
    fC = D == real(D) ? fill(f,n) : fill(f+0im,n)
    for i = 1:length(D)
        fC[i] = Fun(t->exp(D[i]*fC[i](t)),domain(f))
    end
    V*diagm(fC)*inv(V)
end;

Consider the system of coupled harmonic oscillators with periodic parameterized excitation:
     $$x'' + (1+a \cos(2t))x = x-y$$
     $$y'' + (1+a \cos(2t))y = y-x$$
We calculate the Principal Fundamental Matrix $Φ$.

In [4]:
T = π;a=0.15
t = Fun(identity,[0,T])
d=domain(t)
D=Derivative(d)
B=ldirichlet(d)
n=4
A=[ diagm(fill(ldirichlet(d),n));
    D             -I  0I            0I;
   (2+a*cos(2t))   D  -I            0I;
   0I             0I   D            -I;
   -I             0I  (2+a*cos(2t))  D]

Φ = A\eye(n);

It can be further expressed as $Φ(t) = P(t) e^{tB}$ where $P(t)$ is a periodic matrix, and $B$ is constant.
Using the values of $Φ(T)$:

In [5]:
ΦT = Φ(T)

4×4 Array{Float64,2}:
 -0.170879  -0.148885  -0.836059   0.265569
  0.732284  -0.170879  -0.612945  -0.836059
 -0.836059   0.265569  -0.170879  -0.148885
 -0.612945  -0.836059   0.732284  -0.170879

we calculate the matrix $B$ and the Floquet exponents and multipliers.

In [6]:
B = logm(ΦT)/T
D,V = eig(B)

println("Floquet Exponents = \n")
[println(D[i]) for i=1:n]
println("\nFloquet Multipliers = \n")
[println(exp(D[i]*T)) for i=1:n];

Floquet Exponents = 

2.220446049250313e-16 - 0.2683546905319052im
6.10405823109339e-17 + 0.268354690531905im
0.037475319732289236 + 0.9999999999999999im
-0.03747531973228904 + 0.9999999999999997im

Floquet Multipliers = 

0.6651802570066612 - 0.7466828146465897im
0.6651802570066613 + 0.7466828146465889im
-1.1249427991479368 + 6.373407186201634e-16im
-0.888934086921956 + 8.983951009601114e-16im


We can build the matrix exponential $e^{-tB}$ and use it to find the periodic matrix $P(t)$.
Periodicity is numerically confirmed by `Fun` instantiation with low number of coefficients.

In [7]:
PI = Φ*expm(B,-t)
P = Fun(t->PI(t),PeriodicInterval(d))
P|>ncoefficients

223

With the matrix B and the periodic matrix P(t), we can construct 
the solution with any initial conditions for as long as we want!

In [8]:
t = Fun(identity,[0,10T])
d = domain(t)
x0 = rand(n)

PI = Fun(t->P(t),d)
xsol = real(PI*expm(B,t)*x0)

plot(mat(xsol))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 -1 
 
 
 0 
 
 
 1 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 25.6518,96.7241 25.6565,96.6802 25.666,96.5925 25.6803,96.461 25.6993,96.2858 25.723,96.0668 25.7515,95.8043 25.7847,95.4983 25.8227,95.149 25.8654,94.7564 
 25.9129,94.3209 25.9651,93.8425 26.022,93.3215 26.0837,92.7581 26.1501,92.1526 26.2213,91.5052 26.2972,90.8163 26.3779,90.0863 26.4633,89.3153 26.5534,88.5039 
 26.6482,87.6525 26.7478,86.7615 26.8521,85.8314 26.9611,84.8626 27.0749,83.8558 27.1934,82.8116 27.3166,81.7304 27.4445,80.613 27.5772,79.4602 27.7145,78.2725 
 27.8566,77.0508 28.0034,75.7959 28.1549,74.5087 28.3111,73.19 28.472,71.8409 28.6376,70.4623 28.8079,69.0554 28.9829,67.6211 29.1626,66.1607 29.3469,64.6755 
 29.536,63.1666 29.7297,61.6354 29.9282,60.0834 30.1313,58.512 30.3391,56.9226 30.5515,55.317 30.7686,53.6967 30.9904,52.0635 31.2168,50.4191 31.4479,48.7655 
 31.6836,47.1045 31.924,45.4381 32.1691,43.7684 32.4187,42.0975 32.673,40.4276 32.932,38.761 33.1955,37.1 33.4637,35.447 33.7365,33.8046 34.0139,32.1752 
 34.2959,30.5615 34.5826,28.9661 34.8738,27.3918 35.1696,25.8415 35.47,24.3179 35.775,22.8239 36.0846,21.3627 36.3987,19.9371 36.7174,18.5504 37.0407,17.2056 
 37.3685,15.9059 37.7009,14.6545 38.0379,13.4548 38.3793,12.3099 38.7254,11.2233 39.0759,10.1982 39.431,9.23815 39.7905,8.34642 40.1546,7.52644 40.5232,6.78162 
 40.8963,6.11539 41.2739,5.53116 41.656,5.03232 42.0426,4.62227 42.4336,4.30439 42.8291,4.08201 43.2291,3.95846 43.6335,3.93701 44.0424,4.02091 44.4557,4.21334 
 44.8735,4.51745 45.2956,4.93631 45.7222,5.47294 46.1533,6.13028 46.5887,6.9112 47.0285,7.81847 47.4727,8.8548 47.9213,10.0228 48.3743,11.3249 48.8316,12.7636 
 49.2933,14.3411 49.7593,16.0595 50.2297,17.921 50.7045,19.9273 51.1836,22.0803 51.667,24.3816 52.1547,26.8326 52.6467,29.4346 53.143,32.1887 53.6436,35.0959 
 54.1485,38.1568 54.6576,41.3722 55.171,44.7422 55.6887,48.267 56.2107,51.9466 56.7368,55.7804 57.2672,59.7681 57.8018,63.9086 58.3407,68.2008 58.8837,72.6432 
 59.4309,77.2342 59.9823,81.9715 60.5379,86.8528 61.0977,91.8753 61.6616,97.0358 62.2296,102.331 62.8018,107.756 63.3782,113.308 63.9586,118.981 64.5432,124.77 
 65.1318,130.669 65.7246,136.673 66.3214,142.774 66.9223,148.965 67.5273,155.239 68.1363,161.587 68.7493,168.001 69.3664,174.472 69.9875,180.989 70.6126,187.543 
 71.2417,194.123 71.8748,200.718 72.5119,207.315 73.153,213.904 73.798,220.471 74.4469,227.005 75.0998,233.49 75.7566,239.916 76.4173,246.266 77.0819,252.529 
 77.7504,258.689 78.4228,264.733 79.0991,270.647 79.7792,276.417 80.4632,282.028 81.151,287.469 81.8426,292.724 82.5381,297.782 83.2373,302.63 83.9403,307.255 
 84.6472,311.647 85.3577,315.795 86.0721,319.689 86.7901,323.319 87.5119,326.678 88.2374,329.758 88.9667,332.552 89.6996,335.056 90.4362,337.264 91.1764,339.173 
 91.9204,340.781 92.6679,342.086 93.4191,343.089 94.1739,343.789 94.9324,344.19 95.6944,344.292 96.46,344.101 97.2292,343.62 98.0019,342.856 98.7782,341.813 
 99.558,340.499 100.341,338.922 101.128,337.09 101.918,335.011 102.712,332.693 103.509,330.147 104.31,327.382 105.114,324.406 105.922,321.231 106.733,317.867 
 107.547,314.321 108.365,310.605 109.186,306.728 110.011,302.7 110.838,298.529 111.67,294.224 112.504,289.793 113.342,285.246 114.183,280.59 115.027,275.833 
 115.875,270.981 116.726,266.043 117.58,261.024 118.437,255.932 119.298,250.772 120.161,245.551 121.028,240.274 121.898,234.948 122.771,229.578 123.648,224.169 
 124.527,218.728 125.41,213.261 126.295,207.772 127.184,202.269 128.076,196.757 128.97,191.242 129.868,185.732 130.769,180.233 131.673,174.752 132.58,169.297 
 133.489,163.875 134.402,158.495 135.318,153.164 136.236,147.891 137.158,142.685 138.082,137.555 139.01,132.511 139.94,127.561 140.873,122.715 141.808,117.984 
 142.747,113.376 143.688,108.901 144.633,104.57 145.58,100.3